Deep Learning
=============

Assignment 4
------------

Previously in `2_fullyconnected.ipynb` and `3_regularization.ipynb`, we trained fully connected networks to classify [notMNIST](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html) characters.

The goal of this assignment is make the neural network convolutional.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (18724, 28, 28) (18724,)


Reformat into a TensorFlow-friendly shape:
- convolutions need the image data formatted as a cube (width by height by #channels)
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

import numpy as np

def reformat(dataset, labels):
  dataset = dataset.reshape(
    (-1, image_size, image_size, num_channels)).astype(np.float32)
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28, 1) (200000, 10)
Validation set (10000, 28, 28, 1) (10000, 10)
Test set (18724, 28, 28, 1) (18724, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

Let's build a small network with two convolutional layers, followed by one fully connected layer. Convolutional networks are more expensive computationally, so we'll limit its depth and number of fully connected nodes.

In [6]:
num_channels = 1 # grayscale

batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases)
    conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [7]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.854871
Minibatch accuracy: 25.0%
Validation accuracy: 10.0%
Minibatch loss at step 50: 1.646381
Minibatch accuracy: 31.2%
Validation accuracy: 41.1%
Minibatch loss at step 100: 0.757602
Minibatch accuracy: 87.5%
Validation accuracy: 52.3%
Minibatch loss at step 150: 1.246638
Minibatch accuracy: 62.5%
Validation accuracy: 55.0%
Minibatch loss at step 200: 0.841034
Minibatch accuracy: 75.0%
Validation accuracy: 57.9%
Minibatch loss at step 250: 1.053419
Minibatch accuracy: 68.8%
Validation accuracy: 60.5%
Minibatch loss at step 300: 0.405232
Minibatch accuracy: 87.5%
Validation accuracy: 62.1%
Minibatch loss at step 350: 0.206440
Minibatch accuracy: 93.8%
Validation accuracy: 62.1%
Minibatch loss at step 400: 0.561159
Minibatch accuracy: 87.5%
Validation accuracy: 63.3%
Minibatch loss at step 450: 0.633286
Minibatch accuracy: 81.2%
Validation accuracy: 63.5%
Minibatch loss at step 500: 0.801986
Minibatch accuracy: 75.0%
Validation accuracy: 63.1%
M

---
Problem 1
---------

The convolutional model above uses convolutions with stride 2 to reduce the dimensionality. Replace the strides by a max pooling operation (`nn.max_pool()`) of stride 2 and kernel size 2.


In [24]:
# From tf documentation examples:
# 2D convolution, with 'SAME' padding:
# (the output feature map has the same size as the input). 
# Note that {strides} is a 4D array whose shape matches the data layout:
#     [image index, y, x, depth].

To add max pooling, with the 2-stride and 2-size kernel, 
convert the conv layer strides to `[1,1,1,1]`
then add the pool operation:
```python
    pool = tf.nn.max_pool(hidden,
                      ksize=[1, 2, 2, 1],
                      strides=[1, 2, 2, 1],
                      padding='SAME')
``` 

In [30]:
# from https://tensorflow.googlesource.com/tensorflow/+/master/
# tensorflow/models/image/mnist/convolutional.py
# Max pooling. The kernel size spec {ksize} also follows the layout of
# the data. Here we have a pooling window of 2, and a stride of 2.
def conv2d(x, W):
        return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

In [31]:
# Convnet + Maxpoool , 2 stride, kernel size = 2

num_channels = 1 # grayscale

batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

    # Input data.
    tf_train_dataset = tf.placeholder(
        tf.float32, shape=(batch_size, image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Variables.
    layer1_weights = tf.Variable(tf.truncated_normal(
        [patch_size, patch_size, num_channels, depth], stddev=0.1))
    layer1_biases = tf.Variable(tf.zeros([depth]))
    layer2_weights = tf.Variable(tf.truncated_normal(
        [patch_size, patch_size, depth, depth], stddev=0.1))
    layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
    layer3_weights = tf.Variable(tf.truncated_normal(
        [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
    layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    layer4_weights = tf.Variable(tf.truncated_normal(
        [num_hidden, num_labels], stddev=0.1))
    layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.

    def model(data):
        conv = conv2d(data, layer1_weights)
        hidden = tf.nn.relu(conv + layer1_biases)
        pool = tf.nn.max_pool(hidden,
                              ksize=[1, 2, 2, 1],
                              strides=[1, 2, 2, 1],
                              padding='SAME')
        conv = conv2d(pool, layer2_weights)
        hidden = tf.nn.relu(conv + layer2_biases)
        pool = tf.nn.max_pool(hidden,
                          ksize=[1, 2, 2, 1],
                          strides=[1, 2, 2, 1],
                          padding='SAME')
        shape = pool.get_shape().as_list()
        print("shape",shape)
        reshape = tf.reshape(pool, [shape[0], shape[1] * shape[2] * shape[3]])
        print ("reshape", reshape.get_shape())
        hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
        return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
    logits = model(tf_train_dataset)
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
    test_prediction = tf.nn.softmax(model(tf_test_dataset))

shape [16, 7, 7, 16]
reshape (16, 784)
shape [10000, 7, 7, 16]
reshape (10000, 784)
shape [18724, 7, 7, 16]
reshape (18724, 784)


In [32]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3.545891
Minibatch accuracy: 6.2%
Validation accuracy: 8.8%
Minibatch loss at step 50: 2.278292
Minibatch accuracy: 18.8%
Validation accuracy: 21.2%
Minibatch loss at step 100: 0.886302
Minibatch accuracy: 81.2%
Validation accuracy: 44.9%
Minibatch loss at step 150: 1.263465
Minibatch accuracy: 62.5%
Validation accuracy: 50.5%
Minibatch loss at step 200: 0.960826
Minibatch accuracy: 81.2%
Validation accuracy: 58.3%
Minibatch loss at step 250: 0.825721
Minibatch accuracy: 62.5%
Validation accuracy: 60.3%
Minibatch loss at step 300: 0.397241
Minibatch accuracy: 87.5%
Validation accuracy: 63.3%
Minibatch loss at step 350: 0.301645
Minibatch accuracy: 93.8%
Validation accuracy: 63.7%
Minibatch loss at step 400: 0.301141
Minibatch accuracy: 87.5%
Validation accuracy: 64.0%
Minibatch loss at step 450: 0.658283
Minibatch accuracy: 75.0%
Validation accuracy: 63.6%
Minibatch loss at step 500: 0.588159
Minibatch accuracy: 81.2%
Validation accuracy: 66.6%
Min

---
Problem 2
---------

Try to get the best performance you can using a convolutional net. Look for example at the classic [LeNet5](http://yann.lecun.com/exdb/lenet/) architecture, adding Dropout, and/or adding learning rate decay.

---

with
 + dropout: acc score : test: 89.8% /  val 69.1%  / 1300 steps  
 +  L2 regularization   : test: 89.7% / val 68.4%  / 1300 steps 
                        : test: 89.9% / val 68% / 1300 steps
                        : test: 90.8% / val 70.5% / 1800 steps
                        : test: 91.5% / val 71.5 / 2200 steps
 + decaying learning rate ( starting at 0.05, decay rate = 0.95, decay steps 20000)
                        : Test : 90.7% / val: 70.8% / 1800 steps
                        : Test : 91.7% / val: 72.2%  / 2200 steps
 + decaying learning rate ( starting at 0.01, decay rate = 0.95, decay steps 20000)
                        : Test : 87.2% / val: 64.7% / 1800 steps
                        : Test : 88.7% / val: 66.3% / 2600 steps
 + momentum + decaying learning( starting at 0.01, decay rate = 0.95, decay steps 20000)
                        : Test : 91.5% / val: 71.8% / 1800 steps
                        : Test : 92.1% / val: 72.3%  / 2200 steps
                        : Test : 92.4% / val: 72.8%  / 2600 steps
                        : Test : 92.5% / val: 72.0%  / 3000 steps                       

In [75]:
# Convnet + Maxpoool , 2 stride, kernel size = 2

num_channels = 1 # grayscale
beta_reg = 0.001
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

    # Input data.
    tf_train_dataset = tf.placeholder(
        tf.float32, shape=(batch_size, image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Variables.
    layer1_weights = tf.Variable(tf.truncated_normal(
        [patch_size, patch_size, num_channels, depth], stddev=0.1))
    layer1_biases = tf.Variable(tf.zeros([depth]))
    layer2_weights = tf.Variable(tf.truncated_normal(
        [patch_size, patch_size, depth, depth], stddev=0.1))
    layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
    layer3_weights = tf.Variable(tf.truncated_normal(
        [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
    layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    layer4_weights = tf.Variable(tf.truncated_normal(
        [num_hidden, num_labels], stddev=0.1))
    layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.

    def model(data, train = False):
        conv = conv2d(data, layer1_weights)
        hidden = tf.nn.relu(conv + layer1_biases)
        pool = tf.nn.max_pool(hidden,
                              ksize=[1, 2, 2, 1],
                              strides=[1, 2, 2, 1],
                              padding='SAME')
        conv = conv2d(pool, layer2_weights)
        hidden = tf.nn.relu(conv + layer2_biases)
        pool = tf.nn.max_pool(hidden,
                          ksize=[1, 2, 2, 1],
                          strides=[1, 2, 2, 1],
                          padding='SAME')
        shape = pool.get_shape().as_list()
        #print("shape",shape)
        reshape = tf.reshape(pool, [shape[0], shape[1] * shape[2] * shape[3]])
        #print ("reshape", reshape.get_shape())
        hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
        if train:
            hidden = tf.nn.dropout(hidden, 0.5)
            
        return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
    logits = model(tf_train_dataset, train = True)
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
    l2_reg = beta_reg * (tf.nn.l2_loss(layer1_weights) + tf.nn.l2_loss(layer1_biases) + 
                         tf.nn.l2_loss(layer2_weights) + tf.nn.l2_loss(layer2_biases) + 
                         tf.nn.l2_loss(layer3_weights) + tf.nn.l2_loss(layer3_biases) + 
                         tf.nn.l2_loss(layer4_weights) + tf.nn.l2_loss(layer4_biases)) 
    loss += l2_reg
    
    # Optimizer.
    global_step = tf.Variable(0.,)
    learning_rate = tf.train.exponential_decay( 0.01 , #starter_rate
                                                   global_step * batch_size, 
                                                   20000,   # decay every decay_steps
                                                   0.95,     # decay_rate
                                                   staircase=True)
    optimizer = tf.train.MomentumOptimizer(learning_rate, 
                                         0.9).minimize(loss, global_step = global_step)
    #optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
    test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [76]:
num_steps = 3001

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print('Initialized')
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 50 == 0):
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(
            valid_prediction.eval(), valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 4.446920
Minibatch accuracy: 18.8%
Validation accuracy: 10.1%
Minibatch loss at step 50: 2.547412
Minibatch accuracy: 12.5%
Validation accuracy: 8.4%
Minibatch loss at step 100: 2.478847
Minibatch accuracy: 12.5%
Validation accuracy: 18.2%
Minibatch loss at step 150: 2.113735
Minibatch accuracy: 43.8%
Validation accuracy: 29.0%
Minibatch loss at step 200: 1.996222
Minibatch accuracy: 37.5%
Validation accuracy: 39.7%
Minibatch loss at step 250: 1.950966
Minibatch accuracy: 43.8%
Validation accuracy: 49.1%
Minibatch loss at step 300: 1.846483
Minibatch accuracy: 31.2%
Validation accuracy: 58.3%
Minibatch loss at step 350: 1.061053
Minibatch accuracy: 75.0%
Validation accuracy: 57.5%
Minibatch loss at step 400: 1.182891
Minibatch accuracy: 56.2%
Validation accuracy: 59.3%
Minibatch loss at step 450: 1.668632
Minibatch accuracy: 68.8%
Validation accuracy: 63.9%
Minibatch loss at step 500: 1.364868
Minibatch accuracy: 75.0%
Validation accuracy: 63.9%
Mi